In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("./drive/MyDrive/nlp/ensemble/")

In [3]:
! ls

checkpoint-17000  checkpoint-17022  ensemble.ipynb  model  model_4


In [4]:
! pip install sentencepiece transformers datasets python-utils 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 28.9 MB/s 
     |████████████████████████████████| 4.7 MB 55.7 MB/s 
     |████████████████████████████████| 365 kB 68.2 MB/s 
     |████████████████████████████████| 6.6 MB 60.0 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 
     |████████████████████████████████| 101 kB 14.9 MB/s 
     |████████████████████████████████| 212 kB 72.8 MB/s 
     |████████████████████████████████| 115 kB 73.9 MB/s 
     |████████████████████████████████| 141 kB 77.9 MB/s 
     |████████████████████████████████| 127 kB 76.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [34]:
from datasets import load_dataset, load_from_disk, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering  ,T5ForConditionalGeneration
from tqdm import tqdm
from IPython.display import clear_output
import torch

from nltk.translate.bleu_score import sentence_bleu

In [6]:
! ls ./checkpoint-17022

config.json	   scheduler.pt		    trainer_state.json
optimizer.pt	   special_tokens_map.json  training_args.bin
pytorch_model.bin  tokenizer_config.json    vocab.txt
rng_state.pth	   tokenizer.json


# Step be Step

## Answer Predictor

In [7]:
class AnswerPredictor:
  def __init__(self, model, tokenizer, device='cuda', n_best=10, max_length=512, stride=256, no_answer=False):
      """Initializes PyTorch Question Answering Prediction
      It's best to leave use the default values.
      Args:
          model: Fine-tuned torch model
          tokenizer: Transformers tokenizer
          device (torch.device): Running device
          n_best (int): Number of best possible answers
          max_length (int): Tokenizer max length
          stride (int): Tokenizer stride
          no_answer (bool): If True, model can return "no answer"
      """
      self.model = model.eval().to(device)
      self.tokenizer = tokenizer
      self.device = device
      self.max_length = max_length
      self.stride = stride
      self.no_answer = no_answer
      self.n_best = n_best


  def model_pred(self, questions, contexts, batch_size=1):
      n = len(contexts)
      if n%batch_size!=0:
          raise Exception("batch_size must be divisible by sample length")

      tokens = self.tokenizer(questions, contexts, add_special_tokens=True, 
                              return_token_type_ids=True, return_tensors="pt", padding=True, 
                              return_offsets_mapping=True, truncation="only_second", 
                              max_length=self.max_length, stride=self.stride)

      start_logits, end_logits = [], []
      for i in tqdm(range(0, n-batch_size+1, batch_size)):
          with torch.no_grad():
              out = self.model(tokens['input_ids'][i:i+batch_size].to(self.device), 
                          tokens['attention_mask'][i:i+batch_size].to(self.device), 
                          tokens['token_type_ids'][i:i+batch_size].to(self.device))

              start_logits.append(out.start_logits)
              end_logits.append(out.end_logits)

      return tokens, torch.stack(start_logits).view(n, -1), torch.stack(end_logits).view(n, -1)


  def __call__(self, questions, contexts, batch_size=1, answer_max_len=100):
      """Creates model prediction
      
      Args: 
          questions (list): Question strings
          contexts (list): Contexts strings
          batch_size (int): Batch size
          answer_max_len (int): Sets the longests possible length for any answer
        
      Returns:
          dict: The best prediction of the model
              (e.g {0: {"text": str, "score": int}})
      """
      tokens, starts, ends = self.model_pred(questions, contexts, batch_size=batch_size)
      start_indexes = starts.argsort(dim=-1, descending=True)[:, :self.n_best]
      end_indexes = ends.argsort(dim=-1, descending=True)[:, :self.n_best]

      preds = {}
      for i, (c, q) in enumerate(zip(contexts, questions)):  
          min_null_score = starts[i][0] + ends[i][0] # 0 is CLS Token
          start_context = tokens['input_ids'][i].tolist().index(self.tokenizer.sep_token_id)
          
          offset = tokens['offset_mapping'][i]
          valid_answers = []
          for start_index in start_indexes[i]:
              # Don't consider answers that are in questions
              if start_index<start_context:
                  continue
              for end_index in end_indexes[i]:
                  # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                  # to part of the input_ids that are not in the context.
                  if (start_index >= len(offset) or end_index >= len(offset)
                      or offset[start_index] is None or offset[end_index] is None):
                      continue
                  # Don't consider answers with a length that is either < 0 or > max_answer_length.
                  if end_index < start_index or (end_index-start_index+1) > answer_max_len:
                      continue

                  start_char = offset[start_index][0]
                  end_char = offset[end_index][1]
                  valid_answers.append({"score": (starts[i][start_index] + ends[i][end_index]).item(),
                                        "text": c[start_char: end_char]})
                  
          if len(valid_answers) > 0:
              best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
          else:
              best_answer = {"text": "", "score": min_null_score}

          if self.no_answer:
              preds[i] = best_answer if best_answer["score"] >= min_null_score else {"text": "", "score": min_null_score}
          else:
              preds[i] = best_answer

      return preds

In [12]:
# test
context = 'آسمان آبی است'
question = "آسمان چه رنگی است؟"

## ParsBert

In [8]:
! ls ./checkpoint-17022

config.json	   scheduler.pt		    trainer_state.json
optimizer.pt	   special_tokens_map.json  training_args.bin
pytorch_model.bin  tokenizer_config.json    vocab.txt
rng_state.pth	   tokenizer.json


In [9]:
model_path_ParsBERT = "./checkpoint-17022/" 
model_ParsBERT = AutoModelForQuestionAnswering.from_pretrained(model_path_ParsBERT)
tokenizer_ParsBERT = AutoTokenizer.from_pretrained(model_path_ParsBERT)
predictor_ParsBERT = AnswerPredictor(model_ParsBERT, tokenizer_ParsBERT, device='cuda', n_best=10, no_answer=True)

In [46]:
preds_ParsBERT = predictor_ParsBERT([question], [context], batch_size=1)
preds_ParsBERT

100%|██████████| 1/1 [00:00<00:00, 64.34it/s]


{0: {'score': 11.761585235595703, 'text': 'آبی'}}

## mBERT 

In [14]:
! ls ./checkpoint-17000

config.json	   scheduler.pt		    trainer_state.json
optimizer.pt	   special_tokens_map.json  training_args.bin
pytorch_model.bin  tokenizer_config.json    vocab.txt
rng_state.pth	   tokenizer.json


In [37]:
model_path_mBERT = "./checkpoint-17000/" 
model_mBERT = AutoModelForQuestionAnswering.from_pretrained(model_path_mBERT)
tokenizer_mBERT = AutoTokenizer.from_pretrained(model_path_mBERT)
predictor_mBERT = AnswerPredictor(model_mBERT, tokenizer_mBERT, device='cuda', n_best=10, no_answer=True)

In [45]:
preds_mBERT = predictor_mBERT([question], [context], batch_size=1)
preds_mBERT

100%|██████████| 1/1 [00:00<00:00, 80.07it/s]


{0: {'score': 10.958106994628906, 'text': 'آبی'}}

## alBERT

In [18]:
! ls ./model/checkpoint-6796

config.json	   scheduler.pt		    tokenizer.json
optimizer.pt	   special_tokens_map.json  trainer_state.json
pytorch_model.bin  spiece.model		    training_args.bin
rng_state.pth	   tokenizer_config.json


In [22]:
model_path_alBERT = "./model/checkpoint-6796" 
model_alBERT = AutoModelForQuestionAnswering.from_pretrained(model_path_alBERT)
tokenizer_alBERT = AutoTokenizer.from_pretrained(model_path_alBERT)
predictor_alBERT = AnswerPredictor(model_mBERT, tokenizer_alBERT, device='cuda', n_best=10, no_answer=True)

In [44]:
preds_alBERT = predictor_alBERT([question], [context], batch_size=1)
preds_alBERT

100%|██████████| 1/1 [00:00<00:00, 44.11it/s]


{0: {'score': -1.380264401435852, 'text': ' است'}}

## ParsT5

In [24]:
! ls ./model_4

config.json  pytorch_model.bin	      tokenizer_config.json
outs.csv     special_tokens_map.json  tokenizer.json


In [47]:
model_path_ParsT5 = "./model_4" 
device = torch.device('cuda')
model_ParsT5 = T5ForConditionalGeneration.from_pretrained(model_path_ParsT5)
tokenizer_ParsT5 = AutoTokenizer.from_pretrained(model_path_ParsT5)
model_ParsT5.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32103, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32103, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [43]:
input = 'متن: ' + context + '، پرسش: ' + question
input_ids_ParsT5 = tokenizer_ParsT5.encode(input, return_tensors='pt').to(device)
output_ids_ParsT5 = model_ParsT5.generate(input_ids_ParsT5, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
output_ParsT5 = ' '.join([tokenizer_ParsT5.decode(id) for id in output_ids_ParsT5])
pred_ParsT5 = output_ParsT5.replace('<pad>', '').replace('</s>', '').strip()
pred_ParsT5

'آبی'

## Vote

In [72]:
preds_scores = [preds_ParsBERT[0], preds_mBERT[0] , preds_alBERT[0] ]
preds_text = [preds_ParsBERT[0]["text"], preds_mBERT[0]["text"] , preds_alBERT[0]["text"] , pred_ParsT5]
resutls = {}
for pred in preds_text:
  if pred.strip() not in list(resutls.keys()):
    resutls[pred.strip()] = 1
  else:
    resutls[pred.strip()] += 1

In [86]:
high_voted = sorted(resutls)[0]
votes = resutls[high_voted]
if votes==1 :
  high_voted = sorted(preds_scores , key=lambda x: x["score"], reverse=True)[0]['text']

In [87]:
high_voted

'آبی'

# In one function

In [121]:
class AnswerPredictor:
  def __init__(self, model, tokenizer, device='cuda', n_best=10, max_length=512, stride=256, no_answer=False):
      """Initializes PyTorch Question Answering Prediction
      It's best to leave use the default values.
      Args:
          model: Fine-tuned torch model
          tokenizer: Transformers tokenizer
          device (torch.device): Running device
          n_best (int): Number of best possible answers
          max_length (int): Tokenizer max length
          stride (int): Tokenizer stride
          no_answer (bool): If True, model can return "no answer"
      """
      self.model = model.eval().to(device)
      self.tokenizer = tokenizer
      self.device = device
      self.max_length = max_length
      self.stride = stride
      self.no_answer = no_answer
      self.n_best = n_best


  def model_pred(self, questions, contexts, batch_size=1):
      n = len(contexts)
      if n%batch_size!=0:
          raise Exception("batch_size must be divisible by sample length")

      tokens = self.tokenizer(questions, contexts, add_special_tokens=True, 
                              return_token_type_ids=True, return_tensors="pt", padding=True, 
                              return_offsets_mapping=True, truncation="only_second", 
                              max_length=self.max_length, stride=self.stride)

      start_logits, end_logits = [], []
      for i in tqdm(range(0, n-batch_size+1, batch_size)):
          with torch.no_grad():
              out = self.model(tokens['input_ids'][i:i+batch_size].to(self.device), 
                          tokens['attention_mask'][i:i+batch_size].to(self.device), 
                          tokens['token_type_ids'][i:i+batch_size].to(self.device))

              start_logits.append(out.start_logits)
              end_logits.append(out.end_logits)

      return tokens, torch.stack(start_logits).view(n, -1), torch.stack(end_logits).view(n, -1)


  def __call__(self, questions, contexts, batch_size=1, answer_max_len=100):
      """Creates model prediction
      
      Args: 
          questions (list): Question strings
          contexts (list): Contexts strings
          batch_size (int): Batch size
          answer_max_len (int): Sets the longests possible length for any answer
        
      Returns:
          dict: The best prediction of the model
              (e.g {0: {"text": str, "score": int}})
      """
      tokens, starts, ends = self.model_pred(questions, contexts, batch_size=batch_size)
      start_indexes = starts.argsort(dim=-1, descending=True)[:, :self.n_best]
      end_indexes = ends.argsort(dim=-1, descending=True)[:, :self.n_best]

      preds = {}
      for i, (c, q) in enumerate(zip(contexts, questions)):  
          min_null_score = starts[i][0] + ends[i][0] # 0 is CLS Token
          start_context = tokens['input_ids'][i].tolist().index(self.tokenizer.sep_token_id)
          
          offset = tokens['offset_mapping'][i]
          valid_answers = []
          for start_index in start_indexes[i]:
              # Don't consider answers that are in questions
              if start_index<start_context:
                  continue
              for end_index in end_indexes[i]:
                  # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                  # to part of the input_ids that are not in the context.
                  if (start_index >= len(offset) or end_index >= len(offset)
                      or offset[start_index] is None or offset[end_index] is None):
                      continue
                  # Don't consider answers with a length that is either < 0 or > max_answer_length.
                  if end_index < start_index or (end_index-start_index+1) > answer_max_len:
                      continue

                  start_char = offset[start_index][0]
                  end_char = offset[end_index][1]
                  valid_answers.append({"score": (starts[i][start_index] + ends[i][end_index]).item(),
                                        "text": c[start_char: end_char]})
                  
          if len(valid_answers) > 0:
              best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
          else:
              best_answer = {"text": "", "score": min_null_score}

          if self.no_answer:
              preds[i] = best_answer if best_answer["score"] >= min_null_score else {"text": "", "score": min_null_score}
          else:
              preds[i] = best_answer

      return preds

In [112]:
class Ensemble:

  def __init__(self,drive_path = './' ,device='cuda', n_best=10, max_length=512, stride=256, no_answer=True):
    
    # ParsBERT
    self.model_path_ParsBERT = drive_path + f"checkpoint-17022/" 
    self.model_ParsBERT = AutoModelForQuestionAnswering.from_pretrained(self.model_path_ParsBERT)
    self.tokenizer_ParsBERT = AutoTokenizer.from_pretrained(self.model_path_ParsBERT)
    self.predictor_ParsBERT = AnswerPredictor(self.model_ParsBERT, self.tokenizer_ParsBERT, device='cuda', n_best=10, no_answer=True)     

    # mBERT
    self.model_path_mBERT = drive_path + f"checkpoint-17000/" 
    self.model_mBERT = AutoModelForQuestionAnswering.from_pretrained(self.model_path_mBERT)
    self.tokenizer_mBERT = AutoTokenizer.from_pretrained(self.model_path_mBERT)
    self.predictor_mBERT = AnswerPredictor(self.model_mBERT, self.tokenizer_mBERT, device='cuda', n_best=10, no_answer=True)
          
    #alBERT
    self.model_path_alBERT = drive_path + f"model/checkpoint-6796" 
    self.model_alBERT = AutoModelForQuestionAnswering.from_pretrained(self.model_path_alBERT)
    self.tokenizer_alBERT = AutoTokenizer.from_pretrained(self.model_path_alBERT)
    self.predictor_alBERT = AnswerPredictor(self.model_mBERT, self.tokenizer_alBERT, device='cuda', n_best=10, no_answer=True)

    #ParsT%
    self.model_path_ParsT5 = drive_path + f"model_4" 
    device = torch.device('cuda')
    self.model_ParsT5 = T5ForConditionalGeneration.from_pretrained(self.model_path_ParsT5)
    self.tokenizer_ParsT5 = AutoTokenizer.from_pretrained(self.model_path_ParsT5)
    self.model_ParsT5.to(device)


  def pred(self, questions, contexts, batch_size=1):

    high_votes = []
    for (question, context) in zip(questions, contexts):

      preds_ParsBERT = self.predictor_ParsBERT([question], [context], batch_size=1)
      preds_mBERT = self.predictor_mBERT([question], [context], batch_size=1)
      preds_alBERT = self.predictor_alBERT([question], [context], batch_size=1)

      input = 'متن: ' + context + '، پرسش: ' + question
      input_ids_ParsT5 = self.tokenizer_ParsT5.encode(input, return_tensors='pt').to(device)
      output_ids_ParsT5 = self.model_ParsT5.generate(input_ids_ParsT5, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
      output_ParsT5 = ' '.join([self.tokenizer_ParsT5.decode(id) for id in output_ids_ParsT5])
      pred_ParsT5 = output_ParsT5.replace('<pad>', '').replace('</s>', '').strip()

      # votes
      preds_scores = [preds_ParsBERT[0], preds_mBERT[0] , preds_alBERT[0] ]
      preds_text = [preds_ParsBERT[0]["text"], preds_mBERT[0]["text"] , preds_alBERT[0]["text"] , pred_ParsT5]
      resutls = {}
      for pred in preds_text:
        if pred.strip() not in list(resutls.keys()):
          resutls[pred.strip()] = 1
        else:
          resutls[pred.strip()] += 1

      high_voted = sorted(resutls)[0]
      votes = resutls[high_voted]
      if votes==1 :
        high_voted = sorted(preds_scores , key=lambda x: x["score"], reverse=True)[0]['text']

      high_votes.append(high_voted)
    return high_votes
  

In [113]:
e = Ensemble()

In [119]:
contexts = []
questions = []
answers = []

contexts.append('هوا امروز آفتابی است')
questions.append('هوا امروز چگونه است')
answers.append('آفتابی')

contexts.append('خوب، بد، زشت یک فیلم درژانر وسترن اسپاگتی حماسی است که توسط سرجو لئونه در سال ۱۹۶۶ در ایتالیا ساخته شد. زبانی که بازیگران این فیلم به آن تکلم می‌کنند مخلوطی از ایتالیایی و انگلیسی است. این فیلم سومین (و آخرین) فیلم از سه‌گانهٔ دلار (Dollars Trilogy) سرجو لئونه است. این فیلم در حال حاضر در فهرست ۲۵۰ فیلم برتر تاریخ سینما در وب‌گاه IMDB با امتیاز ۸٫۸ از ۱۰، رتبهٔ هشتم را به خود اختصاص داده‌است و به عنوان بهترین فیلم وسترن تاریخ سینمای جهان شناخته می‌شود. «خوب» (کلینت ایستوود، در فیلم، با نام «بلوندی») و «زشت» (ایلای والاک، در فیلم، با نام «توکو») با هم کار می‌کنند و با شگرد خاصی، به گول زدن کلانترهای مناطق مختلف و پول درآوردن از این راه می‌پردازند. «بد» (لی وان کلیف) آدمکشی حرفه‌ای است که به‌خاطر پول حاضر به انجام هر کاری است. «بد»، که در فیلم او را «اِنجل آیز (اِینجل آیز)» (به انگلیسی: Angel Eyes) صدا می‌کنند. به‌دنبال گنجی است که در طی جنگ‌های داخلی آمریکا، به دست سربازی به نام «جکسون»، که بعدها به «کارسون» نامش را تغییر داده، مخفی شده‌است.')
questions.append('در فیلم خوب بد زشت شخصیت ها کجایی صحبت می کنند؟')
answers.append('مخلوطی از ایتالیایی و انگلیسی')

contexts.append('رارداد کرسنت قراردادی برای فروش روزانه معادل ۵۰۰ میلیون فوت مکعب، گاز ترش میدان سلمان است، که در سال ۱۳۸۱ و در زمان وزارت بیژن نامدار زنگنه در دولت هفتم مابین شرکت کرسنت پترولیوم و شرکت ملی نفت ایران منعقد گردید. مذاکرات اولیه این قرارداد از سال ۱۹۹۷ آغاز شد و در نهایت، سال ۲۰۰۱ (۱۳۸۱) به امضای این تفاهم نامه مشترک انجامید. بر اساس مفاد این قرارداد، مقرر شده بود که از سال ۲۰۰۵ با احداث خط لوله در خلیج فارس، گاز فرآورده نشده میدان سلمان (مخزن مشترک با ابوظبی)، به میزان روزانه ۵۰۰ میلیون فوت مکعب (به قول برخی منابع ۶۰۰ میلیون فوت مکعب) به امارات صادر شود. این قرارداد مطابق قوانین داخلی ایران بسته شده‌ و تنها قرارداد نفتی ایران است که از طرف مقابل خود، تضمین گرفته‌است. اجرای این پروژه در سال ۱۳۸۴ با دلایل ارائه شده از سوی دیوان محاسبات ایران از جمله تغییر نیافتن بهای گاز صادراتی و ثابت ماندن آن در هفت سال اول اجرای قرارداد متوقف شد. این در حالی است که طبق تعریف حقوقی، دیوان محاسبات ایران، حق دخالت در قراردادها، پیش از آنکه قراردادها اجرایی و مالی شوند را ندارد.')
questions.append('طرفین قرار داد کرسنت کیا بودن؟	')
answers.append('کرسنت پترولیوم و شرکت ملی نفت ایران')


pred_answers = e.pred(questions,contexts)

100%|██████████| 1/1 [00:00<00:00, 82.24it/s]


In [120]:
pred_answers

['آفتابی',
 'مخلوطی از ایتالیایی و انگلیسی',
 'شرکت کرسنت پترولیوم و شرکت ملی نفت ایران']